<a href="https://colab.research.google.com/github/wonjerry/DCGAN/blob/master/GAN_FashionMnist_focal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/content/gdrive/My Drive/datasets/fmnist/data/", one_hot = True)

# 하이퍼파라미터 설정

total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128

# 플레이스홀더 설정
# Gan은 비지도 학습이므로 오토인코더처럼 Y사용하지 않음
# 노이즈를 입력할 플레이스홀더 Z를 추가

X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

# 생성자 신경망에 사용할 변수설정
# 첫 번째 가중치와 편향은 은닉층 --> 출력하기 위한 변수
# 두 번째 가중치와 편향은 출력층 --> 실제이미지 크기(n_input)와 같아야함

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev = 0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

# 구분자 신경망에 사용할 변수설정
# 은닉층은 생성자와 동일하게 구성
# 구분자는 진짜와 얼마나 가까운가를 판단 0~1사이 값 출력

D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))

# 생성자 신경망 구성
# 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 은닉층 만들고 은닉층에서 실제 이미지와 같은 크기의 결과값을 출력

def generator(noise_z):
    hidden = tf.nn.relu(
                        tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(
                        tf.matmul(hidden, G_W2) + G_b2)
    return output
  
# 구분자 신경망 구성
# 0~1의 스칼라 값을 출력하도록 하였음 ---> sigmoid함수 이용

def discriminator(inputs):
    hidden = tf.nn.relu(
                        tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(
                        tf.matmul(hidden, D_W2) + D_b2)
    return output
  
# 무작위한 노이즈를 만들어주는 유틸리티 함수

def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))
  
# 노이즈 Z를 이용하여 가짜이미지 만들 생성자 G 만들고 
# G가 만든 가짜이미지와 진짜이미니 X를 각각 구분자에 넣어 입력한 이미지가 진짜인지 판별

G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

# 손실값 구하기
# 생성자가 만든 이미지를 구분자가 가짜라고 판단하는 손실값
# 진짜라고 판단하도록 하는 손실값
# 경찰을 학습시키려면 진짜이미지 판별값 D_real은 1에 가까워야하고, 가짜이미지 판별값 D_gene는 0에 가까워야 함

# loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))
# loss_G = tf.reduce_mean(tf.log(D_gene))

# Focal loss function
gamma = 2.0
a = ((1 - D_real)**gamma) * tf.log(D_real)
b = ((1 - (1 - D_gene))**gamma) * tf.log(1 - D_gene)

loss_D = tf.reduce_mean(a + b)
loss_G = tf.reduce_mean(((1 - D_gene)**gamma) * tf.log(D_gene))

##### GAN의 학습은 loss_D와 loss_G 모두를 최대화 하는 것. 둘은 경쟁관계

# 손실값을 이용해 학습
# loss_D 구할 때, 구분자 신경망에 사용되는 변수들만 사용
# loss_G 구할 때, 생성자 신경망에 사용되는 변수들만 사용하여 최적화해야함. 
# 그래야 loss_D를 학습할 때 생성자 변하지 않고, loss_G 학습할 때 구분자 변하지 않기 때문
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]


# 변수 최적화 함수 구성
# 논문에 따르면 loss를 최대화 해야하지만, 최적화에 쓸 수 있는 함수는 minimize뿐이므로 최적화하려는 loss_D와 loss_G에 음수부호 붙임
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)


# 두개의 손실값 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

# 구분자는 X값을, 생성자는 노이즈인 Z값을 받으므로 노이즈를 생성해주는 get_noise를 통해 배치크기만큼 노이즈를 만들고 입력해줌
# 구분자와 생성자 신경망을 각각 학습
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict = {Z: noise})
        
    print('Epoch:', '%04d' % epoch, 'D loss: {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))
    

    
### 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Extracting /content/gdrive/My Drive/datasets/fmnist/data/train-images-idx3-ubyte.gz
Extracting /content/gdrive/My Drive/datasets/fmnist/data/train-labels-idx1-ubyte.gz
Extracting /content/gdrive/My Drive/datasets/fmnist/data/t10k-images-idx3-ubyte.gz
Extracting /content/gdrive/My Drive/datasets/fmnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0000 D loss: -0.2337 G loss: -2.49
Epoch: 0001 D loss: -0.2674 G loss: -2.711
Epoch: 0002 D loss: -1.121 G loss: -1.066
Epoch: 0003 D loss: -0.4979 G loss: -1.896
Epoch: 0004 D loss: -0.4188 G loss: -1.954
Epoch: 0005 D loss: -0.588 G loss: -1.759
Epoch: 0006 D loss: -0.3652 G loss: -2.237
Epoch: 0007 D loss: -0.4368 G loss: -1.93
Epoch: 0008 D loss: -0.5008 G loss: -1.796
Epoch: 0009 D loss: -0.5328 G loss: -1.863
Epoch: 0010 D loss: -0.5221 G loss: -1.875
Epoch: 0011 D loss: -0.6362 G loss: -1.741
Epoch: 0012 D loss: -0.7124 G loss: -1.576
Epoch: 0013 D loss: -0.5213 G loss: -1.897
Epoch: 0014 D loss: -0.5963 G loss: -1.864
Epoch: 0015 D loss: -0.64